In [1]:
PARAMS_COMMON={
    'project_from':'HYBRID-Imbalanced',
    'project_to':'RefinedTreshold-Hybrid-Test'
}

In [2]:
import sys
sys.path.append('/home/ubuntu/intelliscope/modules')
sys.path.append('../..')


#!pip install neptune-client
import os
import neptune

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *
from neptune.sessions import Session
import custom_metrics


NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/'+PARAMS_COMMON['project_from']]
projects=project.get_experiments()



def get_results(model=None, X_pcg=None, X_ecg=None, Y=None):    
    y_pred = model.predict([X_pcg,X_ecg], batch_size=32, verbose=1)
    y = np.argmax(Y,axis=1)
    
    return y, y_pred
    


def neptune_log_metrics(y_test=None, y_pred=None, threshold = 0.5):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)
    

    

In [3]:
project_name=PARAMS_COMMON['project_to']
X_val,Y_val = instead_data_loaders(projects[0].get_parameters(),'val')
X_test,Y_test = instead_data_loaders(projects[0].get_parameters(),'test')

Total  340
97 Images loaded across 2 Categories. Ignored blanks  8
243 Images loaded across 2 Categories. Ignored blanks  0
332 Total After removing blanks
Total  707
199 Images loaded across 2 Categories. Ignored blanks  60
508 Images loaded across 2 Categories. Ignored blanks  9
638 Total After removing blanks


In [4]:
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax



def get_optimal_thres(model, y_val, y_val_pred):
    
    y_val_pred_1=y_val_pred[:,1] 
    fpr, tpr, thresholds = roc_curve(y_val, y_val_pred_1)
    gmeans = sqrt(tpr * (1-fpr))
    ix = argmax(gmeans)
    
    print(f'\nfpr : {fpr[ix]} , tpr : {tpr[ix]} , gmeans : {gmeans[ix]}')
    return thresholds[ix]
    

In [5]:
X_test_pcg = X_test[:,:,:,0:3]/255.0
X_test_ecg = X_test[:,:,:,3:6]/255.0

X_val_pcg = X_val[:,:,:,0:3]/255.0
X_val_ecg = X_val[:,:,:,3:6]/255.0

for i in range(len(projects)):
        
    project    = projects[i]
    print('\n\n',str(project))
    
    model_name = str(project)+'.h5'
    PARAMS     = project.get_parameters()
    PARAMS['tags']=PARAMS['tags'][2:-2].strip().split("', '")+['samplewise']
    
    model = load_model(PARAMS['modelsave_dir']+'/'+model_name)
    
    y_val, y_val_pred=get_results(model, X_val_pcg, X_val_ecg, Y_val)
    
    optimal_thres=get_optimal_thres(model, y_val, y_val_pred)
    
    print('optimal_threshold : ',  optimal_thres, '\n')
    y_test, y_test_pred=get_results(model, X_test_pcg, X_test_ecg, Y_test)
    
    
    sk_metrics=custom_metrics._class_metrics(y_test, y_test_pred[:, 1] > optimal_thres)
    print(sk_metrics)
    gmean=sqrt(sk_metrics['true_negative_rate']*sk_metrics['true_positive_rate'])
    
    PARAMS['G-Mean']=gmean 
    PARAMS['optimal_threshold']=optimal_thres
    
    neptune.init('intelliscope/'+PARAMS_COMMON['project_to'],NEPTUNE_API_TOKEN)
    exp = neptune.create_experiment(name=str(project),description=PARAMS['name'],params=PARAMS,tags=PARAMS['tags'],upload_stdout=True)
    neptune_log_metrics(y_test, y_test_pred, optimal_thres)
    neptune.stop()



 Experiment(HYBIM-1)
332/332 [==============================] - 2s 7ms/sample

fpr : 0.10112359550561797 , tpr : 0.8641975308641975 , gmeans : 0.8813664217090033
optimal_threshold :  0.8902511 

638/638 [==============================] - 4s 6ms/sample
{'accuracy': 0.8322884012539185, 'precision': 0.9355555555555556, 'recall': 0.843687374749499, 'f1_score': 0.8872497365648051, 'f2_score': 0.8605887162714636, 'matthews_corrcoef': 0.5750205154051002, 'cohen_kappa': 0.5634105473196813, 'true_positive_rate': 0.843687374749499, 'true_negative_rate': 0.7913669064748201, 'positive_predictive_value': 0.9355555555555556, 'negative_predictive_value': 0.5851063829787234, 'false_positive_rate': 0.20863309352517986, 'false_negative_rate': 0.156312625250501, 'false_discovery_rate': 0.06444444444444444}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-1


 Experiment(HYBIM-2)
332/332 [==============================] - 1s 4ms/sample

fpr : 0.15730337078651685 , tpr : 0.8065843621399177 , gmeans : 0.8244427955605023
optimal_threshold :  0.89555067 

638/638 [==============================] - 2s 4ms/sample
{'accuracy': 0.7288401253918495, 'precision': 0.8975609756097561, 'recall': 0.7374749498997996, 'f1_score': 0.8096809680968097, 'f2_score': 0.7647547797173732, 'matthews_corrcoef': 0.3749761328990147, 'cohen_kappa': 0.35363839730150737, 'true_positive_rate': 0.7374749498997996, 'true_negative_rate': 0.697841726618705, 'positive_predictive_value': 0.8975609756097561, 'negative_predictive_value': 0.42543859649122806, 'false_positive_rate': 0.302158273381295, 'false_negative_rate': 0.2625250501002004, 'false_discovery_rate': 0.1024390243902439}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-2


 Experiment(HYBIM-3)
332/332 [==============================] - 1s 4ms/sample

fpr : 0.12359550561797752 , tpr : 0.8559670781893004 , gmeans : 0.8661255072841065
optimal_threshold :  0.54046476 

638/638 [==============================] - 2s 4ms/sample
{'accuracy': 0.8369905956112853, 'precision': 0.9006085192697769, 'recall': 0.8897795591182365, 'f1_score': 0.8951612903225806, 'f2_score': 0.891924467657694, 'matthews_corrcoef': 0.5292196873388167, 'cohen_kappa': 0.5290242898312063, 'true_positive_rate': 0.8897795591182365, 'true_negative_rate': 0.6474820143884892, 'positive_predictive_value': 0.9006085192697769, 'negative_predictive_value': 0.6206896551724138, 'false_positive_rate': 0.35251798561151076, 'false_negative_rate': 0.11022044088176353, 'false_discovery_rate': 0.09939148073022312}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


...

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-31


 Experiment(HYBIM-32)
332/332 [==============================] - 2s 7ms/sample

fpr : 0.1797752808988764 , tpr : 0.8518518518518519 , gmeans : 0.8358887162182279
optimal_threshold :  0.9461652 

638/638 [==============================] - 4s 6ms/sample
{'accuracy': 0.8009404388714734, 'precision': 0.8957446808510638, 'recall': 0.843687374749499, 'f1_score': 0.8689370485036121, 'f2_score': 0.8536090835360909, 'matthews_corrcoef': 0.46034708323830015, 'cohen_kappa': 0.4567919443289845, 'true_positive_rate': 0.843687374749499, 'true_negative_rate': 0.6474820143884892, 'positive_predictive_value': 0.8957446808510638, 'negative_predictive_value': 0.5357142857142857, 'false_positive_rate': 0.35251798561151076, 'false_negative_rate': 0.156312625250501, 'false_discovery_rate': 0.10425531914893617}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-32


 Experiment(HYBIM-33)
332/332 [==============================] - 2s 7ms/sample

fpr : 0.07865168539325842 , tpr : 0.8641975308641975 , gmeans : 0.892315492720504
optimal_threshold :  0.791385 

638/638 [==============================] - 4s 6ms/sample
{'accuracy': 0.8166144200626959, 'precision': 0.9098712446351931, 'recall': 0.8496993987975952, 'f1_score': 0.8787564766839377, 'f2_score': 0.861088545897644, 'matthews_corrcoef': 0.5093510847744289, 'cohen_kappa': 0.5043492118298561, 'true_positive_rate': 0.8496993987975952, 'true_negative_rate': 0.697841726618705, 'positive_predictive_value': 0.9098712446351931, 'negative_predictive_value': 0.563953488372093, 'false_positive_rate': 0.302158273381295, 'false_negative_rate': 0.15030060120240482, 'false_discovery_rate': 0.09012875536480687}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF-33
